In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/Code/ntfa_rewrite_batching_fixed/htfatorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os
import webdataset as wds
import torch

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/data/zulqarnain/fmri_data/AVFP/'

task_log_csvs = 'AffVidsNovel_logfiles'

nifti_files = '36p/'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub-%03d_run%01d_36p_mc_MNI_masked.nii.gz'

AFFVIDS_FILE = 'data/avfp_mc_zrest_norest.tar'

tar_file = AFFVIDS_FILE
sink = wds.TarWriter(tar_file)


def affvid_filename(subject, run):
    return affvids_dir + nifti_files + (AFFVID_FILENAME_TEMPLATE % (subject, run))

class TaskElement:
    def __init__(self, task, start, end, run, fear_rating=None):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round((t + TASK_ONSET_DELAY) / 0.8)
                else:
                    return round(t)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)
        self.run = run
        self.fear_rating = fear_rating

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[5])
        end_time = float(cols[6])
        run = int(cols[4])
        fear_rating = float(cols[10])
        yield TaskElement(task, start_time, end_time, run, fear_rating)

def rest_tasks(tasks):
    yield TaskElement('rest', 0, tasks[0].start_time - 1, tasks[0].run)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1
        if tasks[i].run == tasks[i-1].run:
            yield TaskElement('rest', rest_start, rest_end, tasks[i].run)
        else:
            yield TaskElement('rest', rest_start, None, tasks[i-1].run)
            yield TaskElement('rest', 0, rest_end, tasks[i].run)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, tasks[-1].run)

bad_runs = collections.defaultdict(lambda: [])
# bad_runs[3] = [1, 2, 3]
# bad_runs[7] = [3]
# bad_runs[14] = [3]
# bad_runs[22] = [1, 2, 3]
# bad_runs[23] = [1]
# bad_runs[24] = [1, 2, 3]
# bad_runs[26] = [3]

## exclude these to create a mini

exclude_subjects = []
exclude_tasks = ['rest', 'Spiders_11.m4v']

def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0
    with open(task_csv, 'r') as task_csv_file:
        subject = int(task_csv.split('_')[-1].strip('.txt'))
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        rest_lines = [r for r in task_lines if r.task == 'rest']
        rest_lines = sorted(rest_lines, key=lambda t: sentinel(t.run))
        rest_starts_dict = {key: [] for key in range(3, 6)}
        rest_ends_dict = {key: [] for key in range(3, 6)}
        for (i,rest) in enumerate(rest_lines):
            if (rest.run in bad_runs[subject]) or (int(subject) in exclude_subjects):
                continue
            if rest.end_time is not None and rest.start_time is not None:
                rest_ends_dict[rest.run].append(rest.end_time)
                rest_starts_dict[rest.run].append(rest.start_time)
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject] or (int(subject) in exclude_subjects) or (task.task in exclude_tasks):
                logging.info('Excluding block %d %s of run %d for motion', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True, zscore_by_rest=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.rest_start_times = rest_starts_dict[result.run]
            result.rest_end_times = rest_ends_dict[result.run]
            result.individual_differences = {'fear_rating': task.fear_rating}
            yield result

In [5]:
OVERRIDE = True

if not os.path.exists(AFFVIDS_FILE) or OVERRIDE:
    total_trs = 0
    metadata = {
        'blocks': []
    }
    block_id = 0
    for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
        for block in read_tasks(task_csv):
            block.filename = affvid_filename(block.subject, block.run)
#             block.mask = mask_filename(block.subject, block.run)
            block.rest_end_times = '[' + ', '.join(map(str, block.rest_end_times)) + ']'
            block.rest_start_times = '[' + ', '.join(map(str, block.rest_start_times)) + ']'
            block.block = block_id
            block_id += 1
            block.mask = '/data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz'
            block.smooth = 6
            block.load()
            metadata['blocks'].append(block.wds_metadata())

            for vals in block.format_wds():
                sink.write(vals)
            block_trs = (block.end_time - block.start_time)
            total_trs += block_trs
    
    metadata['voxel_locations'] = block.locations
    metadata['num_times'] = total_trs
    torch.save(metadata, tar_file + '.meta')
    logging.info('Recorded metadata, including voxel locations')

sink.close()

09/19/2021 19:00:45 Subject 100
09/19/2021 19:00:45 Excluding block 0 rest of run 3 for motion
09/19/2021 19:00:45 Excluding block 1 rest of run 4 for motion
09/19/2021 19:00:45 Excluding block 2 rest of run 5 for motion
09/19/2021 19:00:45 Block 3 Social_8.mov of run 4 started at 19.000000, ended at 45.000000
09/19/2021 19:00:45 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/36p/sub-100_run4_36p_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
09/19/2021 19:01:06 Block 4 Social_3.mp4 of run 3 started at 21.000000, ended at 47.000000
09/19/2021 19:01:06 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/36p/sub-100_run3_36p_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
09/19/2021 19:01:27 Block 5 Spiders_7.m4v of run 5 started at 21.000000, ended at 47.000000
09/19/2021 19:01:27 Loading Nifti ima

09/19/2021 19:02:53 Excluding block 12 rest of run 5 for motion
09/19/2021 19:02:53 Excluding block 13 rest of run 3 for motion
09/19/2021 19:02:53 Excluding block 14 rest of run 4 for motion
09/19/2021 19:02:53 Block 15 Social_1.m4v of run 5 started at 113.000000, ended at 138.000000
09/19/2021 19:02:54 Block 16 Spiders_12.mp4 of run 3 started at 114.000000, ended at 140.000000
09/19/2021 19:02:54 Block 17 Spiders_10.m4v of run 4 started at 114.000000, ended at 140.000000
09/19/2021 19:02:54 Excluding block 18 rest of run 5 for motion
09/19/2021 19:02:54 Excluding block 19 rest of run 3 for motion
09/19/2021 19:02:54 Excluding block 20 rest of run 4 for motion
09/19/2021 19:02:54 Block 21 Heights_12.m4v of run 4 started at 160.000000, ended at 186.000000
09/19/2021 19:02:54 Block 22 Social_10.mov of run 3 started at 161.000000, ended at 187.000000
09/19/2021 19:02:54 Block 23 Heights_4.m4v of run 5 started at 161.000000, ended at 188.000000
09/19/2021 19:02:54 Excluding block 24 rest 

09/19/2021 19:03:58 Block 33 Heights_12.m4v of run 3 started at 255.000000, ended at 281.000000
09/19/2021 19:03:58 Block 34 Heights_2.mov of run 4 started at 256.000000, ended at 280.000000
09/19/2021 19:03:58 Block 35 Spiders_1.m4v of run 5 started at 256.000000, ended at 282.000000
09/19/2021 19:03:58 Excluding block 36 rest of run 4 for motion
09/19/2021 19:03:58 Excluding block 37 rest of run 3 for motion
09/19/2021 19:03:58 Excluding block 38 rest of run 5 for motion
09/19/2021 19:03:58 Block 39 Heights_10.mp4 of run 3 started at 304.000000, ended at 331.000000
09/19/2021 19:03:58 Block 40 Social_3.mp4 of run 4 started at 304.000000, ended at 330.000000
09/19/2021 19:03:58 Block 41 Social_7.mp4 of run 5 started at 304.000000, ended at 330.000000
09/19/2021 19:03:59 Excluding block 42 rest of run 4 for motion
09/19/2021 19:03:59 Excluding block 43 rest of run 5 for motion
09/19/2021 19:03:59 Excluding block 44 rest of run 3 for motion
09/19/2021 19:03:59 Block 45 Spiders_4.mov of 

09/19/2021 19:05:05 Block 53 Social_3.mp4 of run 3 started at 399.000000, ended at 425.000000
09/19/2021 19:05:05 Excluding block 54 rest of run 4 for motion
09/19/2021 19:05:05 Excluding block 55 rest of run 5 for motion
09/19/2021 19:05:05 Excluding block 56 rest of run 3 for motion
09/19/2021 19:05:05 Block 57 Heights_9.m4v of run 3 started at 445.000000, ended at 471.000000
09/19/2021 19:05:05 Excluding block 58 Spiders_11.m4v of run 4 for motion
09/19/2021 19:05:05 Block 59 Spiders_6.mp4 of run 5 started at 445.000000, ended at 472.000000
09/19/2021 19:05:05 Excluding block 60 rest of run 3 for motion
09/19/2021 19:05:05 Excluding block 61 rest of run 4 for motion
09/19/2021 19:05:05 Excluding block 62 rest of run 5 for motion
09/19/2021 19:05:05 Block 63 Social_11.mp4 of run 4 started at 493.000000, ended at 519.000000
09/19/2021 19:05:06 Block 64 Social_2.m4v of run 3 started at 494.000000, ended at 520.000000
09/19/2021 19:05:06 Block 65 Spiders_9.mp4 of run 5 started at 494.00

09/19/2021 19:06:13 Excluding block 73 rest of run 4 for motion
09/19/2021 19:06:13 Excluding block 74 rest of run 3 for motion
09/19/2021 19:06:13 Subject 107
09/19/2021 19:06:13 Excluding block 0 rest of run 3 for motion
09/19/2021 19:06:13 Excluding block 1 rest of run 4 for motion
09/19/2021 19:06:13 Excluding block 2 rest of run 5 for motion
09/19/2021 19:06:13 Block 3 Spiders_2.mp4 of run 5 started at 18.000000, ended at 43.000000
09/19/2021 19:06:13 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/36p/sub-107_run5_36p_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
09/19/2021 19:06:34 Block 4 Heights_11.mov of run 3 started at 19.000000, ended at 45.000000
09/19/2021 19:06:34 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/36p/sub-107_run3_36p_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz (zscore=True, smooth=6, zscore_by_rest=True

...

09/19/2021 19:51:17 Block 59 Heights_12.m4v of run 4 started at 447.000000, ended at 472.000000
09/19/2021 19:51:17 Excluding block 60 rest of run 5 for motion
09/19/2021 19:51:17 Excluding block 61 rest of run 3 for motion
09/19/2021 19:51:17 Excluding block 62 rest of run 4 for motion
09/19/2021 19:51:17 Block 63 Social_11.mp4 of run 3 started at 493.000000, ended at 519.000000
09/19/2021 19:51:17 Block 64 Spiders_7.m4v of run 4 started at 493.000000, ended at 519.000000
09/19/2021 19:51:17 Block 65 Heights_9.m4v of run 5 started at 494.000000, ended at 520.000000
09/19/2021 19:51:18 Excluding block 66 rest of run 3 for motion
09/19/2021 19:51:18 Excluding block 67 rest of run 4 for motion
09/19/2021 19:51:18 Excluding block 68 rest of run 5 for motion
09/19/2021 19:51:18 Block 69 Heights_3.m4v of run 3 started at 540.000000, ended at 563.000000
09/19/2021 19:51:18 Block 70 Heights_4.m4v of run 4 started at 540.000000, ended at 567.000000
09/19/2021 19:51:18 Block 71 Spiders_12.mp4 o

09/19/2021 19:52:44 Block 4 Heights_3.m4v of run 4 started at 18.000000, ended at 40.000000
09/19/2021 19:52:44 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/36p/sub-148_run4_36p_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
09/19/2021 19:53:05 Block 5 Heights_5.mp4 of run 5 started at 19.000000, ended at 45.000000
09/19/2021 19:53:05 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/36p/sub-148_run5_36p_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/AVFP_mem_mask_threshold90.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
09/19/2021 19:53:25 Excluding block 6 rest of run 4 for motion
09/19/2021 19:53:25 Excluding block 7 rest of run 3 for motion
09/19/2021 19:53:25 Excluding block 8 rest of run 5 for motion
09/19/2021 19:53:25 Block 9 Spiders_2.mp4 of run 3 started at 65.000000, ended at 91.000000
09/19/2021 19:53:25 Block 10 Social_9.mp4 of run 5 started at 65.00

09/19/2021 19:54:34 Excluding block 18 rest of run 3 for motion
09/19/2021 19:54:34 Excluding block 19 rest of run 4 for motion
09/19/2021 19:54:34 Excluding block 20 rest of run 5 for motion
09/19/2021 19:54:34 Block 21 Heights_6.mp4 of run 3 started at 160.000000, ended at 185.000000
09/19/2021 19:54:34 Block 22 Heights_10.mp4 of run 4 started at 161.000000, ended at 188.000000
09/19/2021 19:54:34 Block 23 Spiders_8.mp4 of run 5 started at 161.000000, ended at 187.000000
09/19/2021 19:54:34 Excluding block 24 rest of run 3 for motion
09/19/2021 19:54:34 Excluding block 25 rest of run 5 for motion
09/19/2021 19:54:34 Excluding block 26 rest of run 4 for motion
09/19/2021 19:54:34 Block 27 Spiders_12.mp4 of run 4 started at 208.000000, ended at 234.000000
09/19/2021 19:54:35 Block 28 Heights_1.m4v of run 3 started at 209.000000, ended at 234.000000
09/19/2021 19:54:35 Block 29 Social_7.mp4 of run 5 started at 209.000000, ended at 235.000000
09/19/2021 19:54:35 Excluding block 30 rest o

09/19/2021 19:55:43 Block 39 Heights_8.m4v of run 5 started at 303.000000, ended at 328.000000
09/19/2021 19:55:43 Block 40 Spiders_9.mp4 of run 3 started at 304.000000, ended at 330.000000
09/19/2021 19:55:44 Block 41 Social_7.mp4 of run 4 started at 304.000000, ended at 330.000000
09/19/2021 19:55:44 Excluding block 42 rest of run 5 for motion
09/19/2021 19:55:44 Excluding block 43 rest of run 3 for motion
09/19/2021 19:55:44 Excluding block 44 rest of run 4 for motion
09/19/2021 19:55:44 Block 45 Social_3.mp4 of run 3 started at 350.000000, ended at 376.000000
09/19/2021 19:55:44 Block 46 Social_5.mp4 of run 4 started at 350.000000, ended at 377.000000
09/19/2021 19:55:44 Block 47 Spiders_3.mov of run 5 started at 350.000000, ended at 376.000000
09/19/2021 19:55:44 Excluding block 48 rest of run 3 for motion
09/19/2021 19:55:44 Excluding block 49 rest of run 5 for motion
09/19/2021 19:55:44 Excluding block 50 rest of run 4 for motion
09/19/2021 19:55:44 Block 51 Heights_7.mov of run

09/19/2021 19:56:52 Block 59 Spiders_1.m4v of run 4 started at 447.000000, ended at 472.000000
09/19/2021 19:56:52 Excluding block 60 rest of run 3 for motion
09/19/2021 19:56:52 Excluding block 61 rest of run 5 for motion
09/19/2021 19:56:52 Excluding block 62 rest of run 4 for motion
09/19/2021 19:56:52 Block 63 Heights_8.m4v of run 3 started at 494.000000, ended at 519.000000
09/19/2021 19:56:52 Block 64 Social_11.mp4 of run 4 started at 494.000000, ended at 521.000000
09/19/2021 19:56:53 Block 65 Heights_10.mp4 of run 5 started at 494.000000, ended at 521.000000
09/19/2021 19:56:53 Excluding block 66 rest of run 3 for motion
09/19/2021 19:56:53 Excluding block 67 rest of run 4 for motion
09/19/2021 19:56:53 Excluding block 68 rest of run 5 for motion
09/19/2021 19:56:53 Block 69 Heights_9.m4v of run 4 started at 540.000000, ended at 566.000000
09/19/2021 19:56:53 Block 70 Social_9.mp4 of run 5 started at 540.000000, ended at 566.000000
09/19/2021 19:56:53 Block 71 Heights_12.m4v of

In [6]:
import htfa_torch.tardb as tardb

In [7]:
db = tardb.FmriTarDataset(AFFVIDS_FILE)

In [8]:
db.mean_block(save=True)
db.normalize_activations(save=True)

([tensor(14.1595),
  tensor(7.7991),
  tensor(7.3364),
  tensor(7.7991),
  tensor(14.1595),
  tensor(7.3364),
  tensor(14.1595),
  tensor(7.3364),
  tensor(14.1595),
  tensor(7.7991),
  tensor(7.3364),
  tensor(7.7991),
  tensor(14.1595),
  tensor(7.3364),
  tensor(14.1595),
  tensor(7.3364),
  tensor(7.7991),
  tensor(7.7991),
  tensor(14.1595),
  tensor(7.3364),
  tensor(7.7991),
  tensor(14.1595),
  tensor(7.3364),
  tensor(7.7991),
  tensor(14.1595),
  tensor(7.3364),
  tensor(7.7991),
  tensor(14.1595),
  tensor(7.3364),
  tensor(7.7991),
  tensor(7.3364),
  tensor(14.1595),
  tensor(7.3364),
  tensor(7.7991),
  tensor(14.1595),
  tensor(8.2318),
  tensor(8.6480),
  tensor(13.0705),
  tensor(8.2318),
  tensor(8.6480),
  tensor(13.0705),
  tensor(13.0705),
  tensor(8.2318),
  tensor(8.6480),
  tensor(8.6480),
  tensor(8.2318),
  tensor(13.0705),
  tensor(8.2318),
  tensor(8.6480),
  tensor(13.0705),
  tensor(8.2318),
  tensor(13.0705),
  tensor(8.6480),
  tensor(8.2318),
  tensor(8

In [9]:
logging.info('Finished building TarDb out of AffVids dataset in %s', affvids_dir)

09/19/2021 20:41:18 Finished building TarDb out of AffVids dataset in /data/zulqarnain/fmri_data/AVFP/
